In [1]:
location='/Users/tharakarehan/Desktop/Hand Recognition Final/Train' #change the path according to your location
bklocation='/Users/tharakarehan/Desktop/Hand Recognition Final/background2.jpg'#change the path according to your location
from tkinter import *
from PIL import ImageTk, Image 
from tkinter.filedialog import askopenfilename
from tkinter import simpledialog
from tkinter import messagebox
from tkinter import ttk
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
import cv2
import math
import os
import glob
global IMAGE

def ExtractROI(img1):
    
    #name=Path[13:-4]
    #img1 = cv2.imread(Path, 3)

    h, w,_ = img1.shape
    if h<=w:
        img1 = cv2.rotate(img1, cv2.ROTATE_90_CLOCKWISE)
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        img2=img1.copy()
        img3=img1.copy()
    else:
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        img2=img1.copy()
        img3=img1.copy()
        
    gray = cv2.cvtColor(img1,cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    ret,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    _,contours, hierarchy = cv2.findContours(th,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    max_area=0
    for i in range(len(contours)):
        cnt=contours[i]
        area = cv2.contourArea(cnt)
        if(area>max_area):
            max_area=area
            ci=i
            
    cnt=contours[ci]
    hull = cv2.convexHull(cnt)
    
    
    Blue=[255,0,0]
    Green=[0,255,0]
    Red=[0,0,255]
    White=[255,255,255]
    Black=[0,0,0]
    Yellow=[255,255,0]
    Purple=[128,0,128]
    m=0
    ListC=[Blue,Green,Red,White,Black,Yellow,Purple]

    hull = cv2.convexHull(cnt,returnPoints = False)
    defects = cv2.convexityDefects(cnt,hull)

    mind=0
    maxd=0
    i=0
    FAR=[]
    START=[]
    END=[]
    
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]
        start = tuple(cnt[s][0])
        end = tuple(cnt[e][0])
        far = tuple(cnt[f][0])
        #dist = cv2.pointPolygonTest(cnt,centr,True)
        pt= (100,180)


        # find length of all sides of triangle
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
        s = (a+b+c)/2
        ar = math.sqrt(s*(s-a)*(s-b)*(s-c))

        #distance between point and convex hull
        d=(2*ar)/a

        # apply cosine rule here
        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57


        # ignore angles > 90 and ignore points very close to convex hull(they generally come due to noise)
        if angle <= 90 and d>30:
        
            cv2.circle(img1, far, 25, Red, -1)

            #draw lines around hand
            cv2.line(img1,start, end, [0,255,0], 5)
            m += 1
            FAR.append(far)
            START.append(start)
            END.append(end)
            
        
        
    ListCor1=[]
    ListCor2=[]
    for x_1,y_1 in FAR:
        ListCor1.append([y_1,x_1])
        
    ListCor1.sort()

    for a_1,a_2 in ListCor1[:3]:
        ListCor2.append([a_2,a_1])
    
    ListCor2.sort() 
    #print (ListCor2)
    len1x=int((ListCor2[1][0]-ListCor2[0][0])*4/5)
    len1y=int((ListCor2[1][1]-ListCor2[0][1])*4/5)
    
    dist1= round((len1x**2+len1y**2)**0.5,4)
    
    len2x=int((ListCor2[2][0]-ListCor2[1][0])*4/5)
    len2y=int((ListCor2[2][1]-ListCor2[1][1])*4/5)
    
    dist2= round((len2x**2+len2y**2)**0.5,4)
    
    len3x=int((ListCor1[3][1]-ListCor2[1][0]))
    len3y=int((ListCor1[3][0]-ListCor2[1][1]))
    
    dist3= round((len3x**2+len3y**2)**0.5,4)
    #LenX=max(len1x,len2x)
    #LenY=max(len1y)
    P=round(dist1/dist2,4)
    
    if P>1.2:
        len1x=int((ListCor2[1][0]-ListCor2[0][0])*4/5/P)
        len1y=int((ListCor2[1][1]-ListCor2[0][1])*4/5/P)
    elif P<0.8:
        len1x=int((ListCor2[1][0]-ListCor2[0][0])*4/5/P)
        len1y=int((ListCor2[1][1]-ListCor2[0][1])*4/5/P)
    
    thumb=[(ListCor1[3][1],ListCor1[3][0]),START[0]]
    index=[(ListCor2[2][0]+len2x,ListCor2[2][1]+len2y),START[1]]
    middle=[(int((ListCor2[2][0]+ListCor2[1][0])/2),int((ListCor2[2][1]+ListCor2[1][1])/2)),START[2]]
    ring=[(int((ListCor2[1][0]+ListCor2[0][0])/2),int((ListCor2[1][1]+ListCor2[0][1])/2)),START[3]]
    pinky=[(ListCor2[0][0]-len1x,ListCor2[0][1]-len1y),END[3]]
    
    ThumbLength=round(np.sqrt(np.sum(np.square(np.array([ListCor1[3][1],ListCor1[3][0]])-np.array(START[0])))),4)
    indexLength=round(np.sqrt(np.sum(np.square(np.array([ListCor2[2][0]+len2x,ListCor2[2][1]+len2y])-np.array(START[1])))),4)
    middleLength=round(np.sqrt(np.sum(np.square(np.array([int((ListCor2[2][0]+ListCor2[1][0])/2),int((ListCor2[2][1]+ListCor2[1][1])/2)])-np.array(START[2])))),4)
    ringLength=round(np.sqrt(np.sum(np.square(np.array([int((ListCor2[1][0]+ListCor2[0][0])/2),int((ListCor2[1][1]+ListCor2[0][1])/2)])-np.array(START[3])))),4)
    pinkyLength=round(np.sqrt(np.sum(np.square(np.array([ListCor2[0][0]-len1x,ListCor2[0][1]-len1y])-np.array(END[3])))),4)
    
    Dist1=dist1*5/4
    Dist2=dist2*5/4
    Dist3=dist3
    
    MeatureFeature=np.array([ThumbLength,indexLength,middleLength,ringLength,pinkyLength,Dist1,Dist2,Dist3])/ThumbLength
    
    
    v3=np.array([ListCor2[2][0],ListCor2[2][1]])
    v4=np.array([ListCor2[0][0],ListCor2[0][1]])
    
    theta = np.arctan2((v3-v4)[1], (v3-v4)[0])*180/np.pi
    #print('The rotation of ROI is {:.02f}\u00b0'.format(theta))
    R = cv2.getRotationMatrix2D(tuple(v4),theta,1)
    img_r = cv2.warpAffine(img2,R,(w,h+int(h/2)))
    v1 = (R[:,:2] @ v3 + R[:,-1]).astype(np.int)
    v2 = (R[:,:2] @ v4 + R[:,-1]).astype(np.int)

    d=(v1-v2)[0]//3

    ux = v1[0] + d
    uy = v1[1] + d
    lx = v2[0] - d
    ly = v2[1] + ((v1-v2)[0]+3*d)

    img_c = cv2.cvtColor(img_r, cv2.COLOR_BGR2RGB)
    img_d=img_c.copy()

    cv2.rectangle(img_c, (lx,ly),(ux,uy),(0,255,0),10)
    img_c = cv2.cvtColor(img_c, cv2.COLOR_BGR2RGB)
    
    img_d = cv2.cvtColor(img_d, cv2.COLOR_BGR2RGB)
    roi=img_d[uy:ly,lx:ux,:]
    
    cv2.circle(img3, ring[0], 20, [255,255,255], -1)
    cv2.circle(img3, ring[1], 20, [255,255,255], -1)
    cv2.circle(img3, middle[0], 20, [255,255,255], -1)
    cv2.circle(img3, middle[1], 20, [255,255,255], -1)
    cv2.circle(img3, thumb[0], 20, [255,255,255], -1)
    cv2.circle(img3, thumb[1], 20, [255,255,255], -1)
    cv2.circle(img3, pinky[0], 20, [255,255,255], -1)
    cv2.circle(img3, pinky[1], 20, [255,255,255], -1)
    cv2.circle(img3, index[0], 20, [255,255,255], -1)
    cv2.circle(img3, index[1], 20, [255,255,255], -1)
    
    return [th,img3,img1,img_c,roi,indexLength,pinkyLength,ringLength,middleLength,ThumbLength,MeatureFeature]
 
def ExtractROI1(String):
    
    name=String[:-4]
    img1 = cv2.imread(location+'/'+String, 3)

    h, w,_ = img1.shape
    if h<=w:
        img1 = cv2.rotate(img1, cv2.ROTATE_90_CLOCKWISE)
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        img2=img1.copy()
        img3=img1.copy()
    else:
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        img2=img1.copy()
        img3=img1.copy()
        
    gray = cv2.cvtColor(img1,cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    ret,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    _,contours, hierarchy = cv2.findContours(th,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    max_area=0
    for i in range(len(contours)):
        cnt=contours[i]
        area = cv2.contourArea(cnt)
        if(area>max_area):
            max_area=area
            ci=i
            
    cnt=contours[ci]
    hull = cv2.convexHull(cnt)
    
    
    Blue=[255,0,0]
    Green=[0,255,0]
    Red=[0,0,255]
    White=[255,255,255]
    Black=[0,0,0]
    Yellow=[255,255,0]
    Purple=[128,0,128]
    m=0
    ListC=[Blue,Green,Red,White,Black,Yellow,Purple]

    hull = cv2.convexHull(cnt,returnPoints = False)
    defects = cv2.convexityDefects(cnt,hull)

    mind=0
    maxd=0
    i=0
    FAR=[]
    START=[]
    END=[]
    
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]
        start = tuple(cnt[s][0])
        end = tuple(cnt[e][0])
        far = tuple(cnt[f][0])
        #dist = cv2.pointPolygonTest(cnt,centr,True)
        pt= (100,180)


        # find length of all sides of triangle
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
        s = (a+b+c)/2
        ar = math.sqrt(s*(s-a)*(s-b)*(s-c))

        #distance between point and convex hull
        d=(2*ar)/a

        # apply cosine rule here
        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57


        # ignore angles > 90 and ignore points very close to convex hull(they generally come due to noise)
        if angle <= 90 and d>30:
        
            cv2.circle(img1, far, 25, Red, -1)

            #draw lines around hand
            cv2.line(img1,start, end, [0,255,0], 5)
            m += 1
            FAR.append(far)
            START.append(start)
            END.append(end)
            
        
        
    ListCor1=[]
    ListCor2=[]
    for x_1,y_1 in FAR:
        ListCor1.append([y_1,x_1])
        
    ListCor1.sort()

    for a_1,a_2 in ListCor1[:3]:
        ListCor2.append([a_2,a_1])
    
    ListCor2.sort() 
    #print (ListCor2)
    len1x=int((ListCor2[1][0]-ListCor2[0][0])*4/5)
    len1y=int((ListCor2[1][1]-ListCor2[0][1])*4/5)
    
    dist1= round((len1x**2+len1y**2)**0.5,4)
    
    len2x=int((ListCor2[2][0]-ListCor2[1][0])*4/5)
    len2y=int((ListCor2[2][1]-ListCor2[1][1])*4/5)
    
    dist2= round((len2x**2+len2y**2)**0.5,4)
    
    len3x=int((ListCor1[3][1]-ListCor2[1][0]))
    len3y=int((ListCor1[3][0]-ListCor2[1][1]))
    
    dist3= round((len3x**2+len3y**2)**0.5,4)
    #LenX=max(len1x,len2x)
    #LenY=max(len1y)
    P=round(dist1/dist2,4)
    
    if P>1.2:
        len1x=int((ListCor2[1][0]-ListCor2[0][0])*4/5/P)
        len1y=int((ListCor2[1][1]-ListCor2[0][1])*4/5/P)
    elif P<0.8:
        len1x=int((ListCor2[1][0]-ListCor2[0][0])*4/5/P)
        len1y=int((ListCor2[1][1]-ListCor2[0][1])*4/5/P)
    
    thumb=[(ListCor1[3][1],ListCor1[3][0]),START[0]]
    index=[(ListCor2[2][0]+len2x,ListCor2[2][1]+len2y),START[1]]
    middle=[(int((ListCor2[2][0]+ListCor2[1][0])/2),int((ListCor2[2][1]+ListCor2[1][1])/2)),START[2]]
    ring=[(int((ListCor2[1][0]+ListCor2[0][0])/2),int((ListCor2[1][1]+ListCor2[0][1])/2)),START[3]]
    pinky=[(ListCor2[0][0]-len1x,ListCor2[0][1]-len1y),END[3]]
    
    ThumbLength=round(np.sqrt(np.sum(np.square(np.array([ListCor1[3][1],ListCor1[3][0]])-np.array(START[0])))),4)
    indexLength=round(np.sqrt(np.sum(np.square(np.array([ListCor2[2][0]+len2x,ListCor2[2][1]+len2y])-np.array(START[1])))),4)
    middleLength=round(np.sqrt(np.sum(np.square(np.array([int((ListCor2[2][0]+ListCor2[1][0])/2),int((ListCor2[2][1]+ListCor2[1][1])/2)])-np.array(START[2])))),4)
    ringLength=round(np.sqrt(np.sum(np.square(np.array([int((ListCor2[1][0]+ListCor2[0][0])/2),int((ListCor2[1][1]+ListCor2[0][1])/2)])-np.array(START[3])))),4)
    pinkyLength=round(np.sqrt(np.sum(np.square(np.array([ListCor2[0][0]-len1x,ListCor2[0][1]-len1y])-np.array(END[3])))),4)
    
    Dist1=dist1*5/4
    Dist2=dist2*5/4
    Dist3=dist3
    
    MeatureFeature=np.array([ThumbLength,indexLength,middleLength,ringLength,pinkyLength,Dist1,Dist2,Dist3])/ThumbLength
    
    
    v3=np.array([ListCor2[2][0],ListCor2[2][1]])
    v4=np.array([ListCor2[0][0],ListCor2[0][1]])
    
    theta = np.arctan2((v3-v4)[1], (v3-v4)[0])*180/np.pi
    #print('The rotation of ROI is {:.02f}\u00b0'.format(theta))
    R = cv2.getRotationMatrix2D(tuple(v4),theta,1)
    img_r = cv2.warpAffine(img2,R,(w,h+int(h/2)))
    v1 = (R[:,:2] @ v3 + R[:,-1]).astype(np.int)
    v2 = (R[:,:2] @ v4 + R[:,-1]).astype(np.int)

    d=(v1-v2)[0]//3

    ux = v1[0] + d
    uy = v1[1] + d
    lx = v2[0] - d
    ly = v2[1] + ((v1-v2)[0]+3*d)

    img_c = cv2.cvtColor(img_r, cv2.COLOR_BGR2RGB)
    img_d=img_c.copy()

    cv2.rectangle(img_c, (lx,ly),(ux,uy),(0,255,0),10)
    img_c = cv2.cvtColor(img_c, cv2.COLOR_BGR2RGB)
    
    img_d = cv2.cvtColor(img_d, cv2.COLOR_BGR2RGB)
    roi=img_d[uy:ly,lx:ux,:]
    
    cv2.circle(img3, ring[0], 20, [255,255,255], -1)
    cv2.circle(img3, ring[1], 20, [255,255,255], -1)
    cv2.circle(img3, middle[0], 20, [255,255,255], -1)
    cv2.circle(img3, middle[1], 20, [255,255,255], -1)
    cv2.circle(img3, thumb[0], 20, [255,255,255], -1)
    cv2.circle(img3, thumb[1], 20, [255,255,255], -1)
    cv2.circle(img3, pinky[0], 20, [255,255,255], -1)
    cv2.circle(img3, pinky[1], 20, [255,255,255], -1)
    cv2.circle(img3, index[0], 20, [255,255,255], -1)
    cv2.circle(img3, index[1], 20, [255,255,255], -1)
    
    return [th,img3,img1,img_c,roi,indexLength,pinkyLength,ringLength,middleLength,ThumbLength,MeatureFeature,name]


def open_img():
    global IMAGE 
    # Select the Imagename  from a folder
    x = askopenfilename() 
  
    # opens the image 
    img = Image.open(x) 
    opencvImage = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    IMAGE=cv2.cvtColor(opencvImage, cv2.COLOR_BGR2RGB)
    
    h1, w1,_ = opencvImage.shape
    if h1<=w1:
        opencvImage = cv2.rotate(opencvImage, cv2.ROTATE_90_CLOCKWISE)
        opencvImage = cv2.cvtColor(opencvImage, cv2.COLOR_BGR2RGB)
    else:
        opencvImage = cv2.cvtColor(opencvImage, cv2.COLOR_BGR2RGB)
    
    img=Image.fromarray(opencvImage)
    
    opencvImage = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    th,img3,img1,img_c,roi,idx,pinky,ring,mid,thumb,_=ExtractROI(opencvImage)
    th=Image.fromarray(th)
    img3=Image.fromarray(img3)
    img1=Image.fromarray(img1)
    img_c=Image.fromarray(img_c)
    roi=Image.fromarray(roi)
    
      
    # resize the image and apply a high-quality down sampling filter 
    img = img.resize((180, 240), Image.ANTIALIAS) 
    th  = th.resize((180, 240), Image.ANTIALIAS) 
    img3 = img3.resize((225, 300), Image.ANTIALIAS) 
    img1 = img1.resize((180, 240), Image.ANTIALIAS) 
    img_c = img_c.resize((180, 240), Image.ANTIALIAS) 
    roi = roi.resize((300, 300), Image.ANTIALIAS) 
  
    # PhotoImage class is used to add image to widgets, icons etc 
    #canvas1 = Canvas(gui,height=240, width=180)
    img = ImageTk.PhotoImage(img)
    panel1 = Label(gui, image = img)
    panel1.image = img
    panel1.place(x=300,y=175,anchor='c') 
    #canvas1.create_image(300,175,image=img6)
    
    
    th = ImageTk.PhotoImage(th)
    panel2 = Label(gui, image = th)
    panel2.image = th
    panel2.place(x=500,y=175,anchor='c') 
    
    
    img1 = ImageTk.PhotoImage(img1)
    panel3 = Label(gui, image = img1)
    panel3.image = img1
    panel3.place(x=700,y=175,anchor='c') 
    
    
    img_c = ImageTk.PhotoImage(img_c)
    panel4 = Label(gui, image = img_c)
    panel4.image = img_c
    panel4.place(x=900,y=175,anchor='c') 
    
    
    img3 = ImageTk.PhotoImage(img3)
    panel5 = Label(gui, image = img3)
    panel5.image = img3
    panel5.place(x=400,y=500,anchor='c')
    
    roi = ImageTk.PhotoImage(roi)
    panel6 = Label(gui, image = roi)
    panel6.image = roi
    panel6.place(x=750,y=500,anchor='c')
    
    print ("Done")
    
    #canvas1.place(x=300,y=175,anchor='c')
    
    #canvas1.image = img6
    #canvas1.create_image(300,175,image=img6)
    #canvas1.place(x=300,y=175,anchor='c')
    
    #canvas1.create_image(240,180,image=img,anchor=SE)
    #panel = Label(gui, image = img3) 
      
    # set the image as img  
      
def IMPORT():
    global IMAGE
    #directory='Triplets'
    os.chdir(location)
    print(os.getcwd())
    s=simpledialog.askstring("Input String","Please Enter your name")
    
    ListDir1=os.listdir(location)
    ListDir=[]
    for f in ListDir1:
        if not f.startswith('.'):
            ListDir.append(f[:-4])
    if isinstance(s, str):
        
        if s+'.jpg' in ListDir:
            messagebox.showerror(title="Warning", message="The name "+s+" is already taken. Please use another name",icon='error')
        else:
            filename = s+'.jpg'
            cv2.imwrite(filename,cv2.cvtColor(IMAGE,cv2.COLOR_BGR2RGB))
            messagebox.showinfo(title="Message", message="Successfully Saved into the Database",icon='info')
    
def TRAIN():
    global ListROI
    global ListName
    global ListMeature
    global L
    ListDir1=os.listdir(location)
    ListDir=[]
    for f in ListDir1:
        if not f.startswith('.'):
            ListDir.append(f)
        
    L=len(ListDir)
    print(ListDir)
    
    ListROI=[]
    ListName=[]
    ListMeature=[]
    
    popup = Toplevel()
    popup.geometry("400x100")
    Label(popup, text="TRAINING........").pack()

    progress = 0
    progress_var = DoubleVar()
    progress_bar = ttk.Progressbar(popup, variable=progress_var, maximum=1000)
    progress_bar.pack(ipady=10,ipadx=100)
    popup.pack_slaves()
    
    progress_step = float(1000.0/L)
    for i in range(L):
        _,img3,_,img_c,roi,_,_,_,_,_,MeatureFeature,name=ExtractROI1(str(ListDir[i]))
        ListROI.append(roi)
        ListName.append(name)
        ListMeature.append(MeatureFeature)
        popup.update()
        progress += progress_step
        progress_var.set(progress)
        print("Done_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _"+str(name))
        
    
    popup.destroy()
    
    messagebox.showinfo(title="Message", message="TRAINING COMPLETED!",icon='info')
    
def similarity_score(img1,img2):

    surf=cv2.xfeatures2d.SURF_create()

    # find the keypoints and descriptors with SURF
    kp1, des1 = surf.detectAndCompute(img1,None)
    kp2, des2 = surf.detectAndCompute(img2,None)

    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2, k=2)

    # Apply ratio test
    good = []
    percent=0
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
            a=len(good)
            percent=(a*100)/len(kp2)
    return round(percent,3)
    
def TEST():
    global ROIT
    global FeatureVector
    
    x = askopenfilename() 
  
    # opens the image 
    img = Image.open(x) 
    opencvImage = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    IMAGE=cv2.cvtColor(opencvImage, cv2.COLOR_BGR2RGB)
    
    h1, w1,_ = opencvImage.shape
    if h1<=w1:
        opencvImage = cv2.rotate(opencvImage, cv2.ROTATE_90_CLOCKWISE)
        opencvImage = cv2.cvtColor(opencvImage, cv2.COLOR_BGR2RGB)
    else:
        opencvImage = cv2.cvtColor(opencvImage, cv2.COLOR_BGR2RGB)
    
    img=Image.fromarray(opencvImage)
    
    opencvImage = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    th,img3,img1,img_c,roitest,idx,pinky,ring,mid,thumb,FV=ExtractROI(opencvImage)
    ROIT=roitest
    FeatureVector=FV
    th=Image.fromarray(th)
    img3=Image.fromarray(img3)
    img1=Image.fromarray(img1)
    img_c=Image.fromarray(img_c)
    roitest=Image.fromarray(roitest)
    
      
    # resize the image and apply a high-quality down sampling filter 
    img = img.resize((180, 240), Image.ANTIALIAS) 
    th  = th.resize((180, 240), Image.ANTIALIAS) 
    img3 = img3.resize((225, 300), Image.ANTIALIAS) 
    img1 = img1.resize((180, 240), Image.ANTIALIAS) 
    img_c = img_c.resize((180, 240), Image.ANTIALIAS) 
    roitest = roitest.resize((300, 300), Image.ANTIALIAS) 
  
    # PhotoImage class is used to add image to widgets, icons etc 
    #canvas1 = Canvas(gui,height=240, width=180)
    img = ImageTk.PhotoImage(img)
    panel1 = Label(gui, image = img)
    panel1.image = img
    panel1.place(x=300,y=175,anchor='c') 
    #canvas1.create_image(300,175,image=img6)
    
    
    th = ImageTk.PhotoImage(th)
    panel2 = Label(gui, image = th)
    panel2.image = th
    panel2.place(x=500,y=175,anchor='c') 
    
    
    img1 = ImageTk.PhotoImage(img1)
    panel3 = Label(gui, image = img1)
    panel3.image = img1
    panel3.place(x=700,y=175,anchor='c') 
    
    
    img_c = ImageTk.PhotoImage(img_c)
    panel4 = Label(gui, image = img_c)
    panel4.image = img_c
    panel4.place(x=900,y=175,anchor='c') 
    
    
    img3 = ImageTk.PhotoImage(img3)
    panel5 = Label(gui, image = img3)
    panel5.image = img3
    panel5.place(x=400,y=500,anchor='c')
    
    roitest = ImageTk.PhotoImage(roitest)
    panel6 = Label(gui, image = roitest)
    panel6.image = roitest
    panel6.place(x=750,y=500,anchor='c')
    
    
    label1=Label(gui,bg='lightgreen',text="Relative Finger Lengths", font=("Helvetica", 16))
    label1.place(x=106, y=475,anchor='c')

    label2=Label(gui,bg='lightgreen',text="Index Finger    :  "+str(round(idx/thumb,3)), font=("Helvetica", 14))
    label2.place(x=102, y=525,anchor='c')

    label3=Label(gui,bg='lightgreen',text="Middle Finger  :  "+str(round(mid/thumb,3)), font=("Helvetica", 14))
    label3.place(x=102, y=550,anchor='c')

    label4=Label(gui,bg='lightgreen',text="Ring Finger     :  "+str(round(ring/thumb,3)), font=("Helvetica", 14))
    label4.place(x=102, y=575,anchor='c')

    label5=Label(gui,bg='lightgreen',text="Pinky Finger    :  "+str(round(pinky/thumb,3)), font=("Helvetica", 14))
    label5.place(x=102, y=600,anchor='c')
    
def EVALUATE():
    
    ListDir1=os.listdir(location)
    ListDir=[]
    for f in ListDir1:
        if not f.startswith('.'):
            ListDir.append(f)
            
    LenD=len(ListDir)
    Len=len(ListName)
    ListSimilar=[]
    ListLength=[]
    
    if LenD==L:
        
        for j in range(Len):
            ListSimilar.append(similarity_score(cv2.cvtColor(ListROI[j], cv2.COLOR_RGB2GRAY),cv2.cvtColor(ROIT, cv2.COLOR_RGB2GRAY)))
            ListLength.append(round(np.sqrt(np.sum(np.square(ListMeature[j]-FeatureVector))),3))
 

        def Transform(List):
            Max=max(List)
            Min=min(List)
            return list(map(lambda x:round((x-Max)*100/(Max-Min)+100,3),List))

        for j in range(Len):
            print(str(ListName[j])+(15-len(str(ListName[j])))*' '+str(Transform(ListSimilar)[j])+(8-len(str(Transform(ListSimilar)[j])))*" "+str(Transform(ListLength)[j])+(8-len(str(Transform(ListLength)[j])))*' '+str(round(Transform(ListSimilar)[j]-Transform(ListLength)[j],3)))
        ListDiff=[]
        for j in range(Len):
            ListDiff.append(round(Transform(ListSimilar)[j]-Transform(ListLength)[j],3))
        List1=ListSimilar[:]
        List1.sort()
        Pcount=0
        for h in ListDiff:
            if h>=0:
                Pcount+=1
    
        if max(ListDiff)>=80:
            NAME=str(ListName[ListDiff.index(max(ListDiff))])
            messagebox.showinfo(title="Message", message=NAME,icon='info')
        elif List1[-1]-List1[-2]>=70:
            NAME=str(ListName[ListDiff.index(max(ListSimilar))])
            messagebox.showinfo(title="Message", message=NAME,icon='info')
        elif max(ListDiff)>=60 and float(Pcount/L)<=0.2:
            NAME=str(ListName[ListDiff.index(max(ListDiff))])
            messagebox.showinfo(title="Message", message=NAME,icon='info')
        else:
            messagebox.showinfo(title="Message", message="UNKNOWN",icon='info')
        
        
    else:
        messagebox.showerror(title="Message", message="Train the database first",icon='error')
        
        
def INFO():
    
    def destroy():
        popup.destroy()
        
    ListDir1=os.listdir(location)
    ListDir=[]
    for f in ListDir1:
        if not f.startswith('.'):
            ListDir.append(f)
    LenD=len(ListDir)
    popup = Toplevel()
    popup.geometry("300x300")
    Label(popup, text="There are "+ str(LenD) + " items in the database").pack(pady=8)
    listbox = Listbox(popup)
    listbox.pack(pady=16)
    Button(popup, text='OK', fg='black', bg='red',command=destroy, height=2, width=10).pack()
    count=1
    for k in ListDir:
        listbox.insert(count,ListDir[count-1][:-4])
        count+=1
    listbox.yview()
      
    
def DELETE():
    files = glob.glob(location+'/*')
    for f in files:
        os.remove(f)
    
    messagebox.showerror(title="Message", message="Database cleared successfully!",icon='error')    
    


gui = Tk() 
  

gui.configure(background="light green") 
  
    
gui.title("Hand Recognition System") 
  
    
gui.geometry("1050x700") 

I=Image.open(bklocation)
I=I.resize((1300,850))
background_image=ImageTk.PhotoImage(I)
background_label =Label(gui, image=background_image)
background_label.image=background_image
background_label.place(x=0, y=0, relwidth=1, relheight=1)


canvas1 = Canvas(gui, bg="blue", height=240, width=180)
canvas1.place(x=300,y=175,anchor='c')

canvas2 = Canvas(gui, bg="blue", height=240, width=180)
canvas2.place(x=500,y=175,anchor='c')

canvas3 = Canvas(gui, bg="blue", height=240, width=180)
canvas3.place(x=700,y=175,anchor='c')

canvas4 = Canvas(gui, bg="blue", height=240, width=180)
canvas4.place(x=900,y=175,anchor='c')

canvas5 = Canvas(gui, bg="blue", height=300, width=225)
canvas5.place(x=400,y=500,anchor='c')

canvas6 = Canvas(gui, bg="blue", height=300, width=300)
canvas6.place(x=750,y=500,anchor='c')

label1=Label(gui,bg='black',fg='white',text="HAND RECOGNITION SYSTEM", font=("Helvetica", 30))
label1.place(x=525, y=25,anchor='c')

label2=Label(gui,bg='lightgreen',text="PHOTO", font=("Helvetica", 14))
label2.place(x=300, y=310,anchor='c')

label3=Label(gui,bg='lightgreen',text="SEGMENTATION", font=("Helvetica", 14))
label3.place(x=500, y=310,anchor='c')

label4=Label(gui,bg='lightgreen',text="KEY POINTS DETECTION", font=("Helvetica", 14))
label4.place(x=700, y=310,anchor='c')

label5=Label(gui,bg='lightgreen',text="EXTRACTION OF ROI", font=("Helvetica", 14))
label5.place(x=900, y=310,anchor='c')

label6=Label(gui,bg='lightgreen',text="DETECTION OF FINGERS", font=("Helvetica", 14))
label6.place(x=400, y=665,anchor='c')

label7=Label(gui,bg='lightgreen',text="PALM PRINT", font=("Helvetica", 14))
label7.place(x=750, y=665,anchor='c')


button1 = Button(gui, text='Select Image', fg='black', bg='red',command=open_img,  height=2, width=15) 
button1.place(x=95, y=75, anchor="c")
  
button2 = Button(gui, text='Add to Database', fg='black', bg='red',command=IMPORT, height=2, width=15) 
button2.place(x=95, y=125,anchor='c') 

button3 = Button(gui, text='Train', fg='black', bg='red',command=TRAIN, height=2, width=15) 
button3.place(x=95, y=175,anchor='c')

button4 = Button(gui, text='Select Test Image', fg='black',command=TEST, bg='red', height=2, width=15) 
button4.place(x=95, y=225,anchor='c')

button5 = Button(gui, text='Evaluate', fg='black', bg='red',command=EVALUATE, height=2, width=15) 
button5.place(x=95, y=275,anchor='c')

button6 = Button(gui, text='Database Info', fg='black', bg='red',command=INFO, height=2, width=15) 
button6.place(x=95, y=325,anchor='c')

button7 = Button(gui, text='Clear Database', fg='black', bg='red',command=DELETE, height=2, width=15) 
button7.place(x=95, y=375,anchor='c')

gui.mainloop()

Done


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Applications/anaconda3/envs/dataweekends/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-1-73121fec1aba>", line 663, in EVALUATE
    Len=len(ListName)
NameError: name 'ListName' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Applications/anaconda3/envs/dataweekends/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-1-73121fec1aba>", line 663, in EVALUATE
    Len=len(ListName)
NameError: name 'ListName' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Applications/anaconda3/envs/dataweekends/lib/python3.6/site-packages/PIL/Image.py", line 2638, in open
    fp.seek(0)
AttributeError: 'str' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call

['IMG_003 (2).JPG', 'IMG_161(2).JPG', 'IMG_0083 (1).JPG', 'IMG_162(6).JPG', 'IMG_0065 (7).JPG', 'IMG_0080 (6).JPG', 'IMG_0084 (8).JPG', 'IMG_128(5).JPG', 'IMG_163(5).JPG', 'IMG_002 (5).jpg', 'IMG_001 (3).JPG', 'Tom.jpg', 'IMG_126(8).JPG', 'IMG_118(1).JPG', 'IMG_0087 (7).JPG', 'IMG_007 (4).JPG', 'IMG_0071 (1).JPG', 'IMG_0068 (6).JPG', 'IMG_008 (6).jpg', 'IMG_0082 (3).JPG', 'IMG_127(5).JPG', 'IMG_167(7).JPG', 'IMG_004 (5).JPG']
Done_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _IMG_003 (2)
Done_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _IMG_161(2)
Done_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _IMG_0083 (1)
Done_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _IMG_162(6)
Done_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _IMG_0065 (7)
Done_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _IMG_0080 (6)
Done_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _IMG_0084 (8)
Done_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _IMG_128(5)
Done_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _IMG_163(5)
Done_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _IMG_002 (5)
Done_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _IMG_001 (3)
Done_ _ _ _ _ _ _ _ _ _ _ _ _ _